# ngl::Quaternion

The ```ngl::Quaternion``` class is a 

In [1]:
// You may need to modify these paths to suit
#pragma cling add_library_path("$HOME/NGL/lib")
#ifdef __APPLE__
    #pragma cling load("$HOME/NGL/lib/libNGL.dylib")
#else
    #pragma cling load("$HOME/NGL/lib/libNGL.so")
#endif
#pragma cling add_include_path("$HOME/NGL/include")
#pragma cling add_include_path("$HOME/NGL/gl3w")

In [2]:
#include <ngl/Mat4.h>
#include <ngl/Quaternion.h>
#include <ngl/NGLStream.h> // for printing
#include <iostream>
#include <vector>


To construct an ```ngl::Quaternion``` we can do the following

In [3]:
{
    ngl::Quaternion Quaternion;
    std::cout<<Quaternion<<'\n';
}

0 [0i,0j,0k]


constructor I use the format used in [John Vinces book](https://dl.acm.org/citation.cfm?id=2016678) where we have a scalar and a vector, some libraries do this the other way round and use the w component, make sure you check if using different libs.
```c++
/// @brief  the quaternion data for the scalar real part
Real m_s;
/// @brief  the quaternion data for x
Real m_x;
/// @brief  the quaternion data for y
Real m_y;
/// @brief  the quaternion data for z
Real m_z;
```

We can construct from the components

In [4]:
{
    ngl::Quaternion Quaternion(2.0f,0.0f,1.0f,0.0f);
    std::cout<<Quaternion<<'\n';
}

2 [0i,1j,0k]


Or from a Mat4

In [25]:
{
    ngl::Mat4 tx;
    tx.rotateX(45.0f);
    ngl::Quaternion Quaternion(tx);
    std::cout<<Quaternion<<'\n';
    auto mat=Quaternion.toMat4();
    std::cout<<"We can convert back to matrix form\n";
    std::cout<<mat<<'\n';
}

0.92388 [0.382683i,0j,0k]
We can convert back to matrix form
[1,0,0,0]
[0,0.707107,-0.707107,0]
[0,0.707107,0.707107,0]
[0,0,0,1]



There are also set and get methods.

In [6]:
{
    ngl::Quaternion quat;
    std::cout<<quat<<'\n';
    quat.set(0.2f,1.0f,0.0f,0.0f);
    std::cout<<quat<<'\n';
    quat.setVector(ngl::Vec3(0.0f,0.0f,1.0f));
    std::cout<<quat<<'\n';
    
}

0 [0i,0j,0k]
0.2 [1i,0j,0k]
0.2 [0i,0j,1k]


## Quaternion Mathematics

There are a number of mathematical properties of a quaternion many are described [here](https://en.wikipedia.org/wiki/Quaternion)

```ngl::Quaternion``` supports a number of operations

## Multiplication by a scalar

In [7]:
{
    ngl::Quaternion quat{2.0f,1.0f,0.0f,0.0f};
    std::cout<<quat*0.5f<<'\n';
    quat *= -0.5f;
    std::cout<<quat<<'\n';
}

1 [0.5i,0j,0k]
-1 [-0.5i,-0j,-0k]


   ## Addition and Subtraction

In [10]:
{
    ngl::Quaternion a{2.0f,1.0f,0.0f,1.0f};
    ngl::Quaternion b{3.2f,0.0f,1.0f,0.0f};
    std::cout<<"a-b = "<<a-b<<'\n';
    std::cout<<"a+b ="<<a+b<<'\n';
    a+=b;
    std::cout<<"a+=b "<<a<<'\n';
}

a-b = -1.2 [1i,-1j,1k]
a+b =5.2 [1i,1j,1k]
a+=b 5.2 [1i,1j,1k]


## Multiplication

In [13]:
{
    ngl::Quaternion a{2.0f,1.0f,0.0f,1.0f};
    ngl::Quaternion b{3.2f,0.0f,1.0f,0.0f};
    std::cout<<"a*b = "<<a*b<<'\n';
    std::cout<<"b*a ="<<b*a<<'\n';
    a*=b;
    std::cout<<"a*=b "<<a<<'\n';
}

a*b = 6.4 [2.2i,2j,4.2k]
b*a =6.4 [4.2i,2j,2.2k]
a*=b 6.4 [2.2i,2j,4.2k]


## Other Operations

In [23]:
{
    ngl::Quaternion a{2.0f,1.0f,0.0f,1.0f};
    std::cout<<"Magnitude of a is "<<a.magnitude()<<'\n';
    std::cout<<"Conjugate of a is "<<a.conjugate()<<'\n';
    std::cout<<"Inverse of a is "<<a.inverse()<<'\n';
    std::cout<<"Normalize \n";
    a.normalise();
    std::cout<<a<<'\n';
}

Magnitude of a is 2.44949
Conjugate of a is 2 [-1i,-0j,-1k]
Inverse of a is 2 [-1i,-0j,-1k]
Normalize 
0.816497 [0.408248i,0j,0.408248k]


## Spherical Interpolation

Quaternions are useful for Spherical Interpolation [slerp](https://en.wikipedia.org/wiki/Slerp) This allows use to interpolate from one rotation value to another with a value of  $ 0\leq t \leq 1 $ . Typicall we would set a quaternion for a start rotation and one for the end and use the slerp function to interpolate between them. Usually converting to and from a Mat4 representation for ease, as show in the following example.

In [30]:
{
    ngl::Mat4 start,end;
    start.rotateX(-25.0f);
    end.rotateX(80.2f);
    ngl::Quaternion qstart(start);
    ngl::Quaternion qend(end);
    for(ngl::Real t=0; t<=1.0f; t+=0.1f)
    {
        std::cout<<ngl::Quaternion::slerp(qstart,qend,t)<<'\n';
        std::cout<<"as Matrix \n"<<ngl::Quaternion::slerp(qstart,qend,t).toMat4()<<'\n';
        
    }
}

0.976296 [-0.21644i,0j,0k]
as Matrix 
[1,-0,0,0]
[0,0.906308,0.422618,0]
[-0,-0.422618,0.906308,0]
[0,0,0,1]

0.992027 [-0.126026i,0j,0k]
as Matrix 
[1,-0,0,0]
[0,0.968235,0.250042,0]
[-0,-0.250042,0.968235,0]
[0,0,0,1]

0.999403 [-0.0345507i,0j,0k]
as Matrix 
[1,-0,0,0]
[0,0.997612,0.06906,0]
[-0,-0.06906,0.997612,0]
[0,0,0,1]

0.998362 [0.0572156i,0j,0k]
as Matrix 
[1,0,0,0]
[0,0.993453,-0.114244,0]
[0,0.114244,0.993453,0]
[0,0,0,1]

0.988912 [0.1485i,0j,0k]
as Matrix 
[1,0,0,0]
[0,0.955896,-0.293707,0]
[0,0.293707,0.955896,0]
[0,0,0,1]

0.971134 [0.238533i,0j,0k]
as Matrix 
[1,0,0,0]
[0,0.886204,-0.463296,0]
[0,0.463296,0.886204,0]
[0,0,0,1]

0.945177 [0.326558i,0j,0k]
as Matrix 
[1,0,0,0]
[0,0.78672,-0.617311,0]
[0,0.617311,0.78672,0]
[0,0,0,1]

0.91126 [0.411833i,0j,0k]
as Matrix 
[1,0,0,0]
[0,0.660788,-0.750573,0]
[0,0.750573,0.660788,0]
[0,0,0,1]

0.869667 [0.493638i,0j,0k]
as Matrix 
[1,0,0,0]
[0,0.512642,-0.858602,0]
[0,0.858602,0.512642,0]
[0,0,0,1]

0.82075 [0.571287i,0j,0k]